In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
LR = [round(i*0.0001,4) for i in range(1,8)]
print(f'Coarse Learning rates will be chosen from : {LR}')
lr_tune = 1e-5
SEED = 123
Save = True
batch_size = 10
img_shape = (832,535)
epochs = epochs_train = 30
epochs_tune = 2
val_split  = 0.3 # Training vs. Validation split
class_names = ['Assam_Type','Metal_Sheet','RCC','Vacant']

work_foldr = r'/home/user/Desktop/Backup_E/phd/CNN_Building_classification'

Loss='sparse_categorical_crossentropy' # Loss type 

data_dir = data_dir_train = f'{work_foldr}/Augmented_30000' # Directory of images   

dir_txt = f'{work_foldr}/Hyper_p_R_E.json' # Latest Hyper Params are stored here

past = f'{work_foldr}'# All previous best are stored here
dir_model1 = f'{work_foldr}/model_E.keras'
dir_model = f'{work_foldr}/model_E.weights.h5'# The model's weights are stored here ## Use in conjunction with .save_weights() and .load_weights()

In [ ]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def Init_model():
    """
    This function initializes the custom model and returns it
    Returns: Model (Sequential)
    """
    global img_shape
    global class_names
    classes = len(class_names)

    model = Sequential()

    pretrained_model = tf.keras.applications.InceptionV3(
        include_top=False,
        weights="imagenet",
        input_shape=(img_shape[0],img_shape[1],3),
        pooling='avg',
        classes=classes,
        classifier_activation="softmax",
    )
    
    for layer in pretrained_model.layers:
        layer.trainable = False

    model.add(pretrained_model)
    model.add(Flatten())
    model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Dense(4, activation='softmax'))
    
    
    return model

In [ ]:
def img_aug(data_dir, test_split=0.1):
    """
    This function takes the directory where the image dataset is stored and the optional test split (default= 10%)
    Returns: three values that contain segregated images in the order of (training, validation, testing) datasets
    """
    global SEED, batch_size, val_split, img_shape, class_names
    
    # Calculate the actual validation split considering the test split
    actual_val_split = val_split / (1 - test_split)
    
    # Load the full dataset
    full_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        seed=SEED,
        image_size=img_shape,
        batch_size=None,  # Load without batching first
        class_names=class_names,
    )
    
    # Print dataset size
    print(f"Total number of samples: {tf.data.experimental.cardinality(full_ds).numpy()}")
    
    # Split the dataset into train+val and test
    dataset_size = tf.data.experimental.cardinality(full_ds).numpy()
    train_val_size = int((1 - test_split) * dataset_size)
    test_size = dataset_size - train_val_size
    
    train_val_ds = full_ds.take(train_val_size)
    test_ds = full_ds.skip(train_val_size)
    
    # Split train+val into train and validation
    train_size = int((1 - actual_val_split) * train_val_size)
    val_size = train_val_size - train_size
    
    train_ds = train_val_ds.take(train_size)
    val_ds = train_val_ds.skip(train_size)
    
    # Apply batching
    train_ds = train_ds.batch(batch_size)
    val_ds = val_ds.batch(batch_size)
    test_ds = test_ds.batch(batch_size)
    
    print(f"Train samples: {train_size}")
    print(f"Validation samples: {val_size}")
    print(f"Test samples: {test_size}")
    
    return train_ds, val_ds, test_ds

In [ ]:
def prepare_test_data(test_ds):
    """
    Turns batches to a single array
    Returns: images,labels {List,List} 
    """
    test_images = []
    test_labels = []
    for images, labels in test_ds:
        test_images.append(images.numpy())
        test_labels.append(labels.numpy())
    
    test_images = np.concatenate(test_images, axis=0)
    test_labels = np.concatenate(test_labels, axis=0)
    
    return test_images, test_labels

In [ ]:
# The dictionary that contains all hyperparameters
hyper_params = {
    'lr': 0.0001 ,
    'batch_size': batch_size ,
    'img_shape': img_shape, #(832,535)
}

In [ ]:
# Load and prepare data for accuracy and other scores
train_ds, val_ds, test_ds = img_aug(data_dir)
test_images, test_labels = prepare_test_data(test_ds)

In [ ]:
#The model is loaded and compiled with the various parameters

model = Init_model()
train_ds, val_ds, test_ds = img_aug(data_dir_train)

model.compile(
    optimizer=Adam(learning_rate = hyper_params['lr']),
    loss = Loss,
    metrics=['accuracy'],
    
)
print('Model Setup done successfully')

In [ ]:
# histories keeps all the models trained in this instannce. Can be used for comparative analysis
histories = []

In [ ]:
## Early stopping, Learning rate reduction on plateau and saving of best model (callback.ModelCheckpoint) declared and instantiated here

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)
model_chk_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = dir_model,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [ ]:
#Gradual unfreezing of layers and subtle training

for j in range(-1,-10,-1):
    model.get_layer("inception_v3").layers[j].trainable = True
    if(j<-1):
        model.get_layer("inception_v3").layers[j+1].trainable = False
    print(f'layer {j} unfrozen')
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs_train,  
        callbacks=[early_stopping, lr_scheduler,model_chk_callback]
      )
    histories.append(history)
print('Model Trained Succesfully')

In [ ]:
import matplotlib.pyplot as plt


def display_images_with_labels(dataset, class_names, num_images=9):
    """
    Display images with labels for a peek at the dataset
    """
    plt.figure(figsize=(15, 15))
    
    for images, labels in dataset.take(1):
        for i in range(min(num_images, len(images))):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            
            label = labels[i].numpy()
            class_name = class_names[label]
            
            plt.title(f"Class: {class_name}\nLabel: {label}")
            plt.axis("off")
    
    plt.tight_layout()
    plt.show()

display_images_with_labels(train_ds, class_names)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
import numpy as np

def calculate_metrics(model, test_images, test_labels):
    """
    Used to find accuracy, F1 score and Matthew's correlation Coefficient of the model given as input
    Returns: Accuracy, f1-score, mathhew's correlation coefficient in this order 
    """
    # Make predictions
    predictions = model.predict(test_images)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Convert one-hot encoded labels to class indices if necessary
    if len(test_labels.shape) > 1:
        true_classes = np.argmax(test_labels, axis=1)
    else:
        true_classes = test_labels
    
    # Calculate metrics
    accuracy = accuracy_score(true_classes, predicted_classes)
    f1 = f1_score(true_classes, predicted_classes, average='macro')
    mcc = matthews_corrcoef(true_classes, predicted_classes)
    
    return accuracy, f1, mcc

accuracy, f1, mcc = calculate_metrics(model, test_images, test_labels)

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Matthews Correlation Coefficient: {mcc:.4f}")

In [ ]:
dir_model2 = f'{work_foldr}/Inceptionv3_30000.keras'

In [ ]:
model.save(dir_model2)

In [ ]:
model.load_weights(dir_model)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, confusion_matrix


# This section displays the detailed metrics of the model


def calculate_metrics(model, test_images, test_labels, class_names):
    """
    A more detailed metrics calculator. 
    Returns: overall_accuracy, overall_f1, overall_mcc, per_class_metrics, true_classes, predicted_classes, confusion_matrix
    """
    # Make predictions
    predictions = model.predict(test_images)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Convert one-hot encoded labels to class indices if necessary
    if len(test_labels.shape) > 1:
        true_classes = np.argmax(test_labels, axis=1)
    else:
        true_classes = test_labels
    
    # Calculate overall metrics
    overall_accuracy = accuracy_score(true_classes, predicted_classes)
    overall_f1 = f1_score(true_classes, predicted_classes, average='macro')
    overall_mcc = matthews_corrcoef(true_classes, predicted_classes)
    
    # Calculate per-class metrics
    cm = confusion_matrix(true_classes, predicted_classes)
    n_classes = cm.shape[0]
    per_class_metrics = []
    
    for i in range(n_classes):
        tp = cm[i, i]
        fp = cm[:, i].sum() - tp
        fn = cm[i, :].sum() - tp
        tn = cm.sum() - tp - fp - fn
        
        accuracy = (tp + tn) / (tp + fp + fn + tn) if (tp + fp + fn + tn) > 0 else 0
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        mcc_numerator = (tp * tn - fp * fn)
        mcc_denominator = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
        mcc = mcc_numerator / mcc_denominator if mcc_denominator != 0 else 0
        
        per_class_metrics.append({
            'class': class_names[i],
            'accuracy': accuracy,
            'f1_score': f1,
            'mcc': mcc
        })
    
    return overall_accuracy, overall_f1, overall_mcc, per_class_metrics, true_classes, predicted_classes, cm

def plot_confusion_matrix(cm, class_names):
    """
    Plotter of confusion matrix
    """
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

#Reasserting class_names for ease of access
class_names = ['Assam_Type', 'Metal_Sheet', 'RCC', 'Vacant']

# Assuming `model`, `test_images`, and `test_labels` are declared and defined beforehand
overall_accuracy, overall_f1, overall_mcc, per_class_metrics, true_classes, predicted_classes, cm = calculate_metrics(model, test_images, test_labels, class_names)

# Print overall results
print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Overall F1 Score: {overall_f1:.4f}")
print(f"Overall Matthews Correlation Coefficient: {overall_mcc:.4f}")

# Print per-class results
print("\nPer-class metrics:")
for metrics in per_class_metrics:
    print(f"Class {metrics['class']}:")
    print(f"  Accuracy: {metrics['accuracy']:.4f}")
    print(f"  F1 Score: {metrics['f1_score']:.4f}")
    print(f"  MCC: {metrics['mcc']:.4f}")

# Call the function to plot the confusion matrix
plot_confusion_matrix(cm, class_names)


In [ ]:
"""
Piece of code that plots the validation accuracy trend of all trained models
"""
fig1 = plt.gcf()
accuracy = []
val_accuracy = []
for history in histories:
    history = history.history
    accuracy += history['accuracy']
    val_accuracy += history['val_accuracy']
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.axis(ymin=0.2,ymax=1)
plt.grid()
plt.title(f'Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation']+[i for i in range(len(histories))])
plt.show()
    # i += 1
# plt.legend(['train', 'validation']+[i for i in range(len(histories))])

In [ ]:
"""
Piece of code that plots the validation loss trend of all trained models
"""
loss = []
val_loss = []
for history in histories:
    history = history.history
    loss += history['loss']
    val_loss += history['val_loss']
plt.plot(loss)
plt.plot(val_loss)
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()